In [1]:
'''
Rule Splitting, Uploading to ES
'''
from elasticsearch import Elasticsearch, RequestsHttpConnection
# from requests_aws4auth import AWS4Auth
import numpy as np
import requests #gets urls
# from aws_requests_auth.boto_utils import BotoAWSRequestsAuth

host = 'https://search-lmi-capstone-test-bbewbwsli4uchb7r24zujwrqne.us-east-1.es.amazonaws.com'
# auth = BotoAWSRequestsAuth(aws_host=host,
#                            aws_region='us-east-1',
#                            aws_service='es')
# awsauth = AWS4Auth(YOUR_ACCESS_KEY, YOUR_SECRET_KEY, REGION, 'es')

In [2]:
es = Elasticsearch(host)

# es = Elasticsearch(
#     hosts=[{'host': host, 'port': 443}],
#     use_ssl=True,
#     verify_certs=True,
#     connection_class=RequestsHttpConnection
# )
print(es.info())

# es = Elasticsearch(
#     hosts=[{'host': host, 'port': 443}],
#     http_auth=auth,
#     use_ssl=True,
#     verify_certs=True,
#     connection_class=RequestsHttpConnection
# )
# print(es.info())

{'name': 'dedbbb462bf60acdb6c63621c8bf117c', 'cluster_name': '846033058400:lmi-capstone-test', 'cluster_uuid': 'vhqZptiWQEq0BVgaN6qi8Q', 'version': {'number': '7.8.0', 'build_flavor': 'oss', 'build_type': 'tar', 'build_hash': 'unknown', 'build_date': '2020-10-15T22:09:28.519938Z', 'build_snapshot': False, 'lucene_version': '8.5.1', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'}


## Define Rule Splits

In [3]:

rule_url = "https://mikeanders.org/data/CMS/CMS-2018-0101-0001/Rule/CMS-2018-0101-0001.txt"

def splitRule_headers(rule_url):

    alltxt = requests.get(rule_url).text.lower()#.encode('unicode_escape').decode() #encodes like raw strings
    
    #Isolate Section 2
    initialsplit = alltxt.split("ii. provisions of the proposed regulations") #split before section 2
    sec2andon = initialsplit[1] #choose latter half
    sec2list = sec2andon.split("iii. collection of information requirements") #split before section 3
    splitlist = sec2list[0] #choose first half
    
    rulechunks = {}
    
    startdict = {'a2':['2. proposals for modified participation options under 5-year agreement periods', '3. creating a basic track with glide path to performance-based risk'], \
     'a3':['3. creating a basic track with glide path to performance-based risk', '4. permitting annual participation elections'], \
    'a4b':['b. proposals for permitting election of differing levels of risk within the basic track\'s glide path', 'c. proposals for permitting annual election of beneficiary assignment methodology'], \
    'a4c':['c. proposals for permitting annual election of beneficiary assignment methodology', '5. determining participation options based on medicare ffs revenue and prior participation '], \
    'a5b':['b. differentiating between low revenue acos and high revenue acos', 'c. determining participation options based on prior participation of aco legal entity and aco participants'], \
    'a5c':['c. determining participation options based on prior participation of aco legal entity and aco participants', 'd. monitoring for financial performance'], \
    'a5d': ['d. monitoring for financial performance', '6. requirements for aco participation in two-sided models'], \
    'a6b': ['b. election of msr/mlr by acos', 'c. aco repayment mechanisms'], \
    'a6c': ['c. aco repayment mechanisms', 'd. advance notice for and payment consequences of termination '], \
    'a6d2': ['(2) proposals for advance notice of voluntary termination', '(3) proposals for payment consequences of termination'], \
    'a6d3': ['(3) proposals for payment consequences of termination', '7. participation options for agreement periods beginning in 2019'], \
    'a7b': ['b. methodology for determining financial and quality performance for the 6-month performance years during 2019', 'c. applicability of program policies to acos participating in a 6-month performance year'], \
    'a7c': ['c. applicability of program policies to acos participating in a 6-month performance year', 'b. fee-for-service benefit enhancements'], \
    'b2a': ['a. shared savings program snf 3-day rule waiver', 'b. billing and payment for telehealth services'], \
    'b2b': ['b. billing and payment for telehealth services', 'c. providing tools to strengthen beneficiary engagement'], \
    'c2': ['2. beneficiary incentives', '3. empowering beneficiary choice'], \
    'c3a': ['3. empowering beneficiary choice', 'b. beneficiary opt-in based assignment methodology'], \
    'c3b': ['b. beneficiary opt-in based assignment methodology', 'd. benchmarking methodology refinements'],  \
    'd2': ['2. risk adjustment methodology for adjusting historical benchmark each performance year', '3. use of regional factors when establishing and resetting acos\' benchmarks'], \
    'd3b': ['b. proposals to apply regional expenditures in determining the benchmark for an aco\'s first agreement period', 'c. proposals for modifying the regional adjustment'], \
    'd3c': ['c. proposals for modifying the regional adjustment', 'd. proposals for modifying the methodology for calculating growth rates used in establishing, resetting, and updating the benchmark'], \
    'd3d': ['d. proposals for modifying the methodology for calculating growth rates used in establishing, resetting, and updating the benchmark', '4. technical changes to incorporate references to benchmark rebasing policies'], \
    'd4': ['4. technical changes to incorporate references to benchmark rebasing policies', 'e. updating program policies'], \
    'e2': ['2. revisions to policies on voluntary alignment', '3. revisions to the definition of primary care services used in beneficiary assignment'], \
    'e3': ['3. revisions to the definition of primary care services used in beneficiary assignment', '4. extreme and uncontrollable circumstances policies for the shared savings program'], \
    'e4': ['4. extreme and uncontrollable circumstances policies for the shared savings program', '5. program data and quality measures'], \
    'e5':['5. program data and quality measures', '6. promoting interoperability'], \
    'e6':['6. promoting interoperability', '7. coordination of pharmacy care for aco beneficiaries'], \
    'e7':['7. coordination of pharmacy care for aco beneficiaries', 'f. applicability of proposed policies to track 1+ model acos'], \
    'f2':['2. unavailability of application cycles for entry into the track 1+ model in 2019 and 2020', '3. applicability of proposed policies to track 1+ model acos through revised program regulations or revisions to track 1+ model participation agreements'], \
    'f3':['3. applicability of proposed policies to track 1+ model acos through revised program regulations or revisions to track 1+ model participation agreements', 'g. summary of proposed timing of applicability']}
    
    for key, value in startdict.items():    
       splitlist = splitlist.split(value[0]) #split on start of desired section
       split_further = splitlist[1].split(value[1]) #split again on start of undesired section
       rulechunks[key] = {"text": split_further[0]} #choose only first half to upload to dict
       splitlist = splitlist[1] #choose second half to prepare for next split
    
    #print(rulechunks.keys())   
    #print(rulechunks["f3"])
    
    #lengths = [len(chunk) for chunk in rulechunks.values()]
    #print(lengths) #characters
    
    return rulechunks

In [4]:
def splitRule_chars(rule_url):
    alltxt = requests.get(rule_url).text.lower()#.encode('unicode_escape').decode() #encodes like raw strings
    
    #Isolate Section 2
    initialsplit = alltxt.split("ii. provisions of the proposed regulations") #split before section 2
    sec2andon = initialsplit[1] #choose latter half
    sec2list = sec2andon.split("iii. collection of information requirements") #split before section 3
    splitlist = sec2list[0] #choose first half
    
    #chunk by num chars
    chunklist = [(splitlist[i:i+8000]) for i in range(0, len(splitlist), 8000)]
    
    #make dict
    rulechunks = {}
    keys = range(len(chunklist))
    for i in keys:
        rulechunks[i] = {"text": chunklist[i]}

    return rulechunks

In [5]:
def splitRule_headchars(rule_url):
    chunks = splitRule_headers(rule_url) #call first method
    for key, value in chunks.items():
        if len(value) > 50000: #check to see if chunk is really long
            smalldict = {}
            chunklist = [value[i:i+25000] for i in range(0, len(value), 25000)] #split chunk
            keys = range(len(chunklist))
            for i in keys:
                smalldict[key+str(i)] = chunklist[i] #add to sub-dictionary
            chunks[key] = smalldict #add to big dictionary
        else:
            chunks[key] = {key+str(0): value} #add to big dictionary
    return chunks

In [16]:
def rulesplit_toES(rulechunks, es_index, idnum):
     for key, value in rulechunks.items():
         res = es.index(index=es_index, id=idnum, body={key:value}, doc_type='_doc')
         idnum += 1
     print(f"Last id uploaded: {idnum-1}")

## Uploading Rule Splits to ES

In [53]:
es.indices.refresh(index="summer_test")

{'_shards': {'total': 10, 'successful': 5, 'failed': 0}}

In [54]:
rulesplit_test = splitRule_headers(rule_url)
test_upload = rulesplit_toES(rulesplit_test, "summer_test", 20)

Last id uploaded: 50


In [9]:
split1 = splitRule_headers(rule_url)
res = es.index(index='headers', id=1, body=split1, doc_type='_doc')
res

{'_index': 'headers',
 '_type': '_doc',
 '_id': '1',
 '_version': 2,
 'result': 'updated',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 1,
 '_primary_term': 1}

In [10]:
split2 = splitRule_chars(rule_url)
res2 = es.index(index='char_count', id=2, body=split2, doc_type='_doc')
res2

{'_index': 'char_count',
 '_type': '_doc',
 '_id': '2',
 '_version': 2,
 'result': 'updated',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 1,
 '_primary_term': 1}

In [11]:
split3 = splitRule_headchars(rule_url)
res3 = es.index(index='hybrid', id=3, body=split3, doc_type='_doc')
res3

{'_index': 'hybrid',
 '_type': '_doc',
 '_id': '3',
 '_version': 2,
 'result': 'updated',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 1,
 '_primary_term': 1}

In [12]:
comb = splitRule_headchars(rule_url)
res3 = es.index(index='comb', id=9, body=comb, doc_type='_doc')
res3

{'_index': 'comb',
 '_type': '_doc',
 '_id': '9',
 '_version': 2,
 'result': 'updated',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 1,
 '_primary_term': 1}

In [13]:
comb2 = splitRule_chars(rule_url)
res3 = es.index(index='comb', id=10, body=comb2, doc_type='_doc')
res3

{'_index': 'comb',
 '_type': '_doc',
 '_id': '10',
 '_version': 2,
 'result': 'updated',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 1,
 '_primary_term': 1}

## Test Queries Below

In [44]:
summer_test_search = es.search(index="summer_test", doc_type="_doc", body={"query": {"simple_query_string": {"query": "medicare"}}})

In [66]:
summer_test_query = es.search(index="summer_test", doc_type="_doc", body={"query": {"match": {"text": {"query": "the"}}}})

In [67]:
summer_test_query

{'took': 6,
 'timed_out': False,
 '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [65]:
print("total hits:", len(summer_test_search["hits"]["hits"]))

total hits: 10


In [50]:
#summer_test_search

In [18]:
es.indices.refresh(index="headers")

{'_shards': {'total': 10, 'successful': 5, 'failed': 0}}

In [50]:
res_test = es.search(index="headers", doc_type="_doc", body={"query": {"match": {"text": "medicare"}}})
#print("%d documents found" % res_test['hits']['total'])
for doc in res_test['hits']['hits']:
    print("%s) %s" % (doc['_id'], doc['_source']['content']))

res_test

{'took': 3,
 'timed_out': False,
 '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [20]:
query_body = {  "query": {"match":{'text':'participation'}}}

result = es.search(index="comb", body=query_body, size=999)
print("total hits:", len(result["hits"]["hits"]))
result

total hits: 0


In [63]:
res_test = es.search(index="comb", doc_type="_doc", body={"query": {"simple_query_string": {"query": "medicare"}}})

In [ ]:
#res_test